## Symbols

In [1]:
x = :a
typeof(x)

Symbol

In [2]:
x = :something
typeof(x)

Symbol

## Expressions

In [3]:
ex = :(a+b)
typeof(ex)

Expr

In [4]:
ex.args

3-element Array{Any,1}:
 :+
 :a
 :b

In [5]:
ex.head

:call

### Interpolation in expressions

Sometimes it is useful to be able to make a substitution inside an expression.

Instead of using an abstract symbol like `X` we can substitute whatever is stored in `X`  using `$X`. Notice that variable `X` needs to be defined in the scope of the expression (otherwise there will be an error) and it can have a value or even another expression.


In [6]:
ex = :(X + b)

:(X + b)

We can substitute `X` for what is stored in `X`

In [7]:
X = 1;
ex = :($X + b)

:(1 + b)

This is valid even if `X` is an `Expr`

In [8]:
X = :(x+y);
ex = :($X + b)

:((x + y) + b)

## Manipulating expressions

In [9]:
ex = :(a+b)

:(a + b)

In [10]:
a = 1
b = 1
eval(ex)

2

Let us consider the following problem:

Given an expression `ex`, change all symbols `:a` by `:(2*a)`.


We can substitue `a` by `2*a` inside the expression `ex`

In [11]:
ex.args[2] = :(2*a)

:(2a)

In [12]:
ex

:(2a + b)

In [13]:
a = 1
b = 1
eval(ex)

3

Notice that in the previous cells he have "manually accessed" position 2 of the arguments of the expression. We have updated `ex.args[2]` directly because we knew that Symbol `a` was in position 2.

What if we want to change Symbol `:a` in an expression independently of where is located?


In [14]:
ex = :(a + b + a)

:(a + b + a)

In [15]:
ex.args[2] = :(2*a)

:(2a)

In [16]:
ex

:(2a + b + a)

In [17]:
ex.args[4] = :(2*a)

:(2a)

In [18]:
ex

:(2a + b + 2a)

We can also make a for loop that iterates over the arguments

In [19]:
ex = :(a + b + a)

:(a + b + a)

In [20]:
for (i,arg) in enumerate(ex.args)
    println(i,"\t", arg,"\t", typeof(arg))
end

1	+	Symbol
2	a	Symbol
3	b	Symbol
4	a	Symbol


In [21]:
for (i,arg) in enumerate(ex.args)
    if arg == :a
        ex.args[i] = :(2*a)
    end
end

In [22]:
ex

:(2a + b + 2a)

#### Expressions inside expressions 

What happens if an expression has more expressions inside? 

In this case we never find symbol `:a` inside the expression `ex=:(2a + b + 2a)` simply iterating over the arguments. We don't find it because the arguments now are `[+, :(2a), :b, :(2a)]`  and `:(2a)` is an expression.




In [23]:
ex

:(2a + b + 2a)

In [24]:
for (i,arg) in enumerate(ex.args)
    println(i,"\t", arg,"\t", typeof(arg))
end

1	+	Symbol
2	2a	Expr
3	b	Symbol
4	2a	Expr


In [25]:
for (i,arg) in enumerate(ex.args)
    if arg == :a
        print("I have found :a ")
        ex.args[i] = :(2*a)
    end
end

In [26]:
ex

:(2a + b + 2a)

## Finding Symbols inside expressions inside expressions inside...

Expressions can contain other expressions which can contain even more expressions...

The problem of substituting `:a` by `:2a` in any expression `ex` requires us to iterate over any "subexpression" contained in the arguments of `ex`.


https://mikeinnes.github.io/MacroTools.jl/stable/pattern-matching/#Expression-Walking-1


In [27]:
ex

:(2a + b + 2a)

In [28]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 2
        3: Symbol a
    3: Symbol b
    4: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 2
        3: Symbol a


In [29]:
ex.args

4-element Array{Any,1}:
 :+   
 :(2a)
 :b   
 :(2a)

In [30]:
ex = :(2a + b + 2a)

:(2a + b + 2a)

In [31]:
global symbols =[]

function get_all_symbols(expression::Expr)
    for arg in expression.args
        if typeof(arg)==Expr
            get_all_symbols(arg)
        end
        if typeof(arg)==Symbol
            push!(symbols, arg)
        end
    end
end

get_all_symbols (generic function with 1 method)

In [32]:
get_all_symbols(ex)

In [33]:
symbols

6-element Array{Any,1}:
 :+
 :*
 :a
 :b
 :*
 :a

In [34]:
x = :a

:a

In [35]:
typeof(ex.args[3]) == Symbol

true

In [36]:
# change any a by c
function change_a_by_c(ex::Expr)
    for i in 1:length(ex.args)
        arg = ex.args[i]
        
        if typeof(arg) == Expr
            change_a_by_c(arg)
        elseif typeof(arg) == Symbol
            if arg == :a
                ex.args[i] = :c 
            end
        end
    end
end

change_a_by_c (generic function with 1 method)

In [37]:
ex = :(2a + b + 2a)
change_a_by_c(ex)
ex

:(2c + b + 2c)

In [38]:
ex = :(2a + b + 2*(a+ (b*a)/2))
change_a_by_c(ex)
ex

:(2c + b + 2 * (c + (b * c) / 2))

### Apply a change to a Symbol given by a function : NOT WORKING NOW

Let us consider we have some transformation `f` that takes a `Symbol` (or an expression) that we call `s` and returns either a symbol or an expression.

Now we would like to make a funciton that everytime symbol `s` is found it is updated by `f(s)`.

In [93]:
function apply_f_to_s!(ex::Tex, pattern::Tp, substitution::Ts)

    if typeof(ex) != Expr
        return f(ex)
    else
        ex = f(ex)
        #println("\t", ex, ex.args)
        for i in 1:length(ex.args)
            ex.args[i]  = apply_f_to_s!(ex.args[i], f) 
            #ex.args[i] = arg
        end
    end
end

TypeError: TypeError: in Type{...} expression, expected UnionAll, got typeof(apply_f_to_s!)

In [89]:
function f(s)
    if s==:(a^2)
        return :(2*a)
    else
        return s
    end 
end

f (generic function with 1 method)

In [90]:
ex1 = :(2 + a^2+ b + c )
println(ex1)
apply_f_to_s!(ex1, f)
println(ex1)

2 + a ^ 2 + b + c
2 + nothing + b + c


In [84]:
ex1 = :(2 + (a^2+ b + a^2) + c + a^2 )
println(ex1)
apply_f_to_s!(ex1, f)
println(ex1)

2 + (a ^ 2 + b + a ^ 2) + c + a ^ 2
	2 + (a ^ 2 + b + a ^ 2) + c + a ^ 2Any[:+, 2, :(a ^ 2 + b + a ^ 2), :c, :(a ^ 2)]
	a ^ 2 + b + a ^ 2Any[:+, :(a ^ 2), :b, :(a ^ 2)]
	2aAny[:*, 2, :a]
	2aAny[:*, 2, :a]
	2aAny[:*, 2, :a]
2 + nothing + c + nothing


In [448]:
global symbols =[]
function substitution_into_expression(expression::Expr, input::Union{Expr, Symbol}, output::Union{Expr, Symbol})
    for arg in expression.args
        if typeof(arg)==Expr
            substitution_into_expression
    end
end

LoadError: syntax: incomplete: "function" at none:1 requires end

In [184]:
using MacroTools

In [185]:
ex

:(2a + b + 2a)

In [205]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 2
        3: Symbol a
    3: Symbol b
    4: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 2
        3: Symbol a


## Splatting interpolation

## Nested quote